<a href="https://colab.research.google.com/github/wesha-904/Local-Food-Wastage-Management-System/blob/main/food_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name** - Local Food Wastage Management System


# **GitHub Link -**

[Anwesha's Github](https://github.com/wesha-904/Local-Food-Wastage-Management-System)

# **Problem Statement**


Food wastage is a significant issue, with many households and restaurants discarding surplus food while numerous people struggle with food insecurity. This project aims to develop a Local Food Wastage Management System, where:
- Restaurants and individuals can list surplus food.
- NGOs or individuals in need can claim the food.
- SQL stores available food details and locations.
- A Streamlit app enables interaction, filtering, CRUD operation and visualization.




# Business Use Cases


- Connecting surplus food providers to those in need through a structured platform.
- Reducing food waste by redistributing excess food efficiently.
-Enhancing accessibility via geolocation features to locate food easily.
-Data analysis on food wastage trends for better decision-making.

# 1. Data Preparation


In [693]:
import pandas as pd
import sqlite3

In [694]:
# Load data from CSV files
receivers = pd.read_csv("https://raw.githubusercontent.com/wesha-904/Local-Food-Wastage-Management-System/refs/heads/main/receivers_data.csv")
providers = pd.read_csv("https://raw.githubusercontent.com/wesha-904/Local-Food-Wastage-Management-System/refs/heads/main/providers_data.csv")
food_listings = pd.read_csv("https://raw.githubusercontent.com/wesha-904/Local-Food-Wastage-Management-System/refs/heads/main/food_listings_data.csv")
claims = pd.read_csv("https://raw.githubusercontent.com/wesha-904/Local-Food-Wastage-Management-System/refs/heads/main/claims_data.csv")

#### Data Preprocessing

In [695]:
providers

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577
...,...,...,...,...,...,...
995,996,"Vasquez, Ruiz and Flowers",Restaurant,"84308 Justin Stravenue\nNew Amberside, NE 53447",Williamview,+1-319-378-7627x0682
996,997,Garza-Williams,Catering Service,"08864 Figueroa Radial Suite 948\nJennaberg, AZ...",East Rossside,001-924-441-3963x746
997,998,Novak Group,Grocery Store,"934 Zachary Run\nMelissamouth, WY 02729",Joshuastad,(903)642-1969x3300
998,999,Moody Ltd,Grocery Store,"17580 Ernest Hills\nLake Michaelmouth, OR 56416",Stevenchester,637.300.3664x4880


In [696]:
print("\nNumber of null values per column:")
print(providers.isnull().sum())

# On inspection there are no NULL values


Number of null values per column:
Provider_ID    0
Name           0
Type           0
Address        0
City           0
Contact        0
dtype: int64


In [697]:
providers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Provider_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   Address      1000 non-null   object
 4   City         1000 non-null   object
 5   Contact      1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [698]:
# Identify numerical and categorical features
numerical_features_providers = providers.select_dtypes(include=['int64', 'float64']).columns
categorical_features_providers = providers.select_dtypes(include=['object']).columns

# Results to see the identified columns
print("Numerical Features:", numerical_features_providers)
print("Categorical Features:", categorical_features_providers)

Numerical Features: Index(['Provider_ID'], dtype='object')
Categorical Features: Index(['Name', 'Type', 'Address', 'City', 'Contact'], dtype='object')


In [699]:
# Count duplicate rows
duplicate_count = providers.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [700]:
# Standardized contact column
import re

def standardize_phone(phone):
    # Remove all non-digit characters except 'x' for extension
    digits = re.sub(r'\D', '', phone)

    # Keep only first 10 digits for the main number
    main_number = digits[-10:]

    # Format: +1-XXX-XXX-XXXX
    formatted = f"+1-{main_number[0:3]}-{main_number[3:6]}-{main_number[6:]}"

    return formatted

# Apply to 'contact' column
providers['Contact'] = providers['Contact'].astype(str).apply(standardize_phone)

# Check result
print(providers['Contact'].head())


0    +1-600-220-0480
1    +1-838-901-6297
2    +1-517-295-2206
3    +1-944-893-5401
4    +1-193-714-6577
Name: Contact, dtype: object


In [701]:
providers

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-838-901-6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,+1-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,+1-944-893-5401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,+1-193-714-6577
...,...,...,...,...,...,...
995,996,"Vasquez, Ruiz and Flowers",Restaurant,"84308 Justin Stravenue\nNew Amberside, NE 53447",Williamview,+1-787-627-0682
996,997,Garza-Williams,Catering Service,"08864 Figueroa Radial Suite 948\nJennaberg, AZ...",East Rossside,+1-441-396-3746
997,998,Novak Group,Grocery Store,"934 Zachary Run\nMelissamouth, WY 02729",Joshuastad,+1-421-969-3300
998,999,Moody Ltd,Grocery Store,"17580 Ernest Hills\nLake Michaelmouth, OR 56416",Stevenchester,+1-003-664-4880


In [702]:
# Cleaning and standardizing the 'Name' column by removing extra spaces, unwanted characters, and converting all text to lowercase for consistency
import re
import pandas as pd

def clean_name_lower(name):
    if pd.isna(name):
        return name
    # Remove leading/trailing spaces
    name = name.strip()
    # Collapse multiple spaces
    name = re.sub(r'\s+', ' ', name)
    # Remove digits and special chars except space, hyphen, apostrophe
    name = re.sub(r"[^a-zA-Z\s\-']", "", name)
    # Convert to lowercase
    name = name.lower()
    return name

# Apply to Name column
providers['Name'] = providers['Name'].apply(clean_name_lower)

# Check results
print(providers['Name'].head(10))


0              gonzales-cochran
1    nielsen johnson and fuller
2                  miller-black
3     clark prince and williams
4                coleman-farley
5                lawson-walters
6                     ortiz-lee
7                  curtis-lewis
8                    nguyen inc
9      hall nguyen and martinez
Name: Name, dtype: object


In [703]:
print(providers['Type'].unique())

['Supermarket' 'Grocery Store' 'Restaurant' 'Catering Service']


In [704]:
# Cleaning and standardizing the 'City' column
import re

def clean_city(name):
    if pd.isna(name):
        return name
    # Remove leading/trailing spaces
    name = name.strip()
    # Collapse multiple spaces
    name = re.sub(r'\s+', ' ', name)
    # Remove unwanted characters except letters, spaces, and hyphens
    name = re.sub(r"[^a-zA-Z\s\-]", "", name)
    # Convert to lowercase for consistency
    name = name.lower()
    return name

# Apply cleaning to the 'City' column
providers['City'] = providers['City'].apply(clean_city)

# Check unique values after cleaning
print(providers['City'].unique())


['new jessica' 'east sheena' 'lake jesusview' 'mendezmouth'
 'valentineside' 'shannonside' 'lake christopherburgh' 'washingtonville'
 'tinamouth' 'west cherylfort' 'south robert' 'east williamburgh'
 'nicolefort' 'garciamouth' 'west theresaberg' 'west vanessafort'
 'west dawn' 'cassandraville' 'south melanieshire' 'east samantha'
 'port corystad' 'port lisamouth' 'bradleyborough' 'hestermouth'
 'anitashire' 'west adammouth' 'new natasha' 'jessestad' 'patrickmouth'
 'west pamelaborough' 'south kellyberg' 'jameschester' 'janetborough'
 'north garybury' 'andersonmouth' 'cindyshire' 'new amanda' 'perkinsbury'
 'brittanyville' 'jennifertown' 'bentleyburgh' 'east nicholasbury'
 'south karen' 'jamesville' 'east deniseborough' 'south lisaberg'
 'east andrewland' 'timothyview' 'lake traceyburgh' 'west danielborough'
 'rebeccabury' 'port dianaberg' 'lake allen' 'lake rachael' 'danachester'
 'michaelview' 'port robin' 'north kevinhaven' 'marissaville' 'wilsonport'
 'lake joelshire' 'marymouth' 'm

In [705]:
providers.head()

,Provider_ID,Name,Type,Address,City,Contact
0,1,gonzales-cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",new jessica,+1-600-220-0480
1,2,nielsen johnson and fuller,Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",east sheena,+1-838-901-6297
2,3,miller-black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",lake jesusview,+1-517-295-2206
3,4,clark prince and williams,Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",mendezmouth,+1-944-893-5401
4,5,coleman-farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",valentineside,+1-193-714-6577


In [706]:
receivers

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154
...,...,...,...,...,...
995,996,Matthew Curtis,Shelter,Lanechester,127-889-4442x1289
996,997,Amanda Cain,NGO,New Steven,+1-001-491-5601x5316
997,998,Theodore Briggs,Individual,South Sandra,930-609-9442x5031
998,999,Cheyenne Ramsey,NGO,Lake Jeffery,001-326-320-4816x15300


In [707]:
print("\nNumber of null values per column:")
print(receivers.isnull().sum())

# On inspection there are no NULL values


Number of null values per column:
Receiver_ID    0
Name           0
Type           0
City           0
Contact        0
dtype: int64


In [708]:
receivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Receiver_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   City         1000 non-null   object
 4   Contact      1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [709]:
# Count duplicate rows
duplicate_count = receivers.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [710]:
# Standardized contact column
import re

def standardize_phone(phone):
    # Remove all non-digit characters except 'x' for extension
    digits = re.sub(r'\D', '', phone)

    # Keep only first 10 digits for the main number
    main_number = digits[-10:]

    # Format: +1-XXX-XXX-XXXX
    formatted = f"+1-{main_number[0:3]}-{main_number[3:6]}-{main_number[6:]}"

    return formatted

# Apply to 'contact' column
receivers['Contact'] = receivers['Contact'].astype(str).apply(standardize_phone)

# Check result
print(receivers['Contact'].head())


0    +1-955-922-5295
1    +1-761-042-1570
2    +1-023-009-4856
3    +1-829-649-1111
4    +1-067-491-0154
Name: Contact, dtype: object


In [711]:
# Cleaning and standardizing the 'Name' column by removing extra spaces, unwanted characters,
# replacing hyphens with spaces, and converting all text to lowercase for consistency
import re
import pandas as pd

def clean_name_lower(name):
    if pd.isna(name):
        return name
    # Remove leading/trailing spaces
    name = name.strip()
    # Replace hyphens with spaces
    name = name.replace('-', ' ')
    # Collapse multiple spaces
    name = re.sub(r'\s+', ' ', name)
    # Remove digits and special chars except space and apostrophe
    name = re.sub(r"[^a-zA-Z\s']", "", name)
    # Convert to lowercase
    name = name.lower()
    return name

# Apply to Name column
receivers['Name'] = receivers['Name'].apply(clean_name_lower)

# Check results
print(receivers['Name'].head(10))


0        donald gomez
1        laurie ramos
2        ashley mckee
3          erika rose
4         john romero
5    mandy sutton phd
6       kenneth baker
7         james perez
8        emily turner
9        mary salazar
Name: Name, dtype: object


In [712]:
print(receivers['Type'].unique())

['Shelter' 'Individual' 'NGO' 'Charity']


In [713]:
# Cleaning and standardizing the 'City' column
import re

def clean_city(name):
    if pd.isna(name):
        return name
    # Remove leading/trailing spaces
    name = name.strip()
    # Collapse multiple spaces
    name = re.sub(r'\s+', ' ', name)
    # Remove unwanted characters except letters, spaces, and hyphens
    name = re.sub(r"[^a-zA-Z\s\-]", "", name)
    # Convert to lowercase for consistency
    name = name.lower()
    return name

# Apply cleaning to the 'City' column
receivers['City'] = receivers['City'].apply(clean_city)

# Check unique values after cleaning
print(receivers['City'].unique())

['port carlburgh' 'lewisburgh' 'south randalltown' 'south shaneville'
 'bakerport' 'east sharimouth' 'south edwinborough' 'benjaminburgh'
 'west robert' 'wrightland' 'south gregorymouth' 'katherineside'
 'lake matthewstad' 'burkeside' 'port ronald' 'port tara' 'silvaport'
 'moranhaven' 'north cynthiaberg' 'sandrahaven' 'jonesside' 'lake nicole'
 'aliciabury' 'monicafort' 'lake elizabeth' 'port matthewmouth'
 'susanfurt' 'courtneyfurt' 'lake kendra' 'north heather' 'spencermouth'
 'taylorfort' 'walshfort' 'north robert' 'south rachaelhaven' 'olsonville'
 'west erik' 'josephview' 'moorechester' 'west carrie' 'port kellyburgh'
 'port dawntown' 'amberfort' 'davisborough' 'south laurachester'
 'east teresahaven' 'east saraport' 'south tonyaborough' 'north sarah'
 'port johnchester' 'hornemouth' 'tomburgh' 'higginsmouth' 'larastad'
 'jasonshire' 'new mary' 'michaelton' 'lake mary' 'phillipborough'
 'south kathleenbury' 'north catherinefurt' 'east william'
 'north darinshire' 'port belinda' '

In [714]:
food_listings

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner
...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,3/30/2025,467,Catering Service,Phillipsfort,Vegan,Breakfast
996,997,Fish,22,3/18/2025,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast
997,998,Fruits,6,3/22/2025,444,Restaurant,New Billy,Non-Vegetarian,Dinner
998,999,Pasta,15,3/30/2025,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch


In [715]:
print("\nNumber of null values per column:")
print(food_listings.isnull().sum())

# On inspection there are no NULL values


Number of null values per column:
Food_ID          0
Food_Name        0
Quantity         0
Expiry_Date      0
Provider_ID      0
Provider_Type    0
Location         0
Food_Type        0
Meal_Type        0
dtype: int64


In [716]:
food_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Food_ID        1000 non-null   int64 
 1   Food_Name      1000 non-null   object
 2   Quantity       1000 non-null   int64 
 3   Expiry_Date    1000 non-null   object
 4   Provider_ID    1000 non-null   int64 
 5   Provider_Type  1000 non-null   object
 6   Location       1000 non-null   object
 7   Food_Type      1000 non-null   object
 8   Meal_Type      1000 non-null   object
dtypes: int64(3), object(6)
memory usage: 70.4+ KB


In [717]:
# Count duplicate rows
duplicate_count = food_listings.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [718]:
print(food_listings['Food_Name'].unique())

['Bread' 'Soup' 'Fruits' 'Vegetables' 'Dairy' 'Rice' 'Pasta' 'Salad'
 'Chicken' 'Fish']


In [719]:
print(food_listings['Food_Type'].unique())

['Non-Vegetarian' 'Vegan' 'Vegetarian']


In [720]:
print(food_listings['Meal_Type'].unique())

['Breakfast' 'Dinner' 'Lunch' 'Snacks']


In [721]:
print(food_listings['Provider_Type'].unique())

['Grocery Store' 'Catering Service' 'Restaurant' 'Supermarket']


In [722]:
print(food_listings.columns.tolist())


['Food_ID', 'Food_Name', 'Quantity', 'Expiry_Date', 'Provider_ID', 'Provider_Type', 'Location', 'Food_Type', 'Meal_Type']


In [723]:
food_listings['Expiry_Date']

,Expiry_Date
0,3/17/2025
1,3/24/2025
2,3/28/2025
3,3/16/2025
4,3/19/2025
...,...
995,3/30/2025
996,3/18/2025
997,3/22/2025
998,3/30/2025


In [724]:
food_listings['Expiry_Date']= pd.to_datetime(food_listings['Expiry_Date'], errors='coerce').dt.strftime('%Y-%m-%d')

print(food_listings['Expiry_Date'].head())

0    2025-03-17
1    2025-03-24
2    2025-03-28
3    2025-03-16
4    2025-03-19
Name: Expiry_Date, dtype: object


In [725]:
claims

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17
...,...,...,...,...,...
995,996,855,211,Completed,3/13/2025 19:40
996,997,980,746,Cancelled,3/17/2025 22:04
997,998,832,967,Cancelled,3/13/2025 18:00
998,999,917,90,Completed,3/1/2025 15:31


In [726]:
print("\nNumber of null values per column:")
print(claims.isnull().sum())

# On inspection there are no NULL values


Number of null values per column:
Claim_ID       0
Food_ID        0
Receiver_ID    0
Status         0
Timestamp      0
dtype: int64


In [727]:
claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Claim_ID     1000 non-null   int64 
 1   Food_ID      1000 non-null   int64 
 2   Receiver_ID  1000 non-null   int64 
 3   Status       1000 non-null   object
 4   Timestamp    1000 non-null   object
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


In [728]:
# Count duplicate rows
duplicate_count = claims.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [729]:
print(claims['Status'].unique())

['Pending' 'Cancelled' 'Completed']


In [730]:
# Convert Timestamp column in claims to standardized datetime
claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], errors='coerce')

# Optional: format as string YYYY-MM-DD HH:MM:SS
claims['Timestamp'] = claims['Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

print(claims['Timestamp'].head())

0    2025-03-05 05:26:00
1    2025-03-11 10:24:00
2    2025-03-21 00:59:00
3    2025-03-04 09:08:00
4    2025-03-14 15:17:00
Name: Timestamp, dtype: object


In [731]:
claims

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00
2,3,626,492,Completed,2025-03-21 00:59:00
3,4,61,933,Cancelled,2025-03-04 09:08:00
4,5,345,229,Pending,2025-03-14 15:17:00
...,...,...,...,...,...
995,996,855,211,Completed,2025-03-13 19:40:00
996,997,980,746,Cancelled,2025-03-17 22:04:00
997,998,832,967,Cancelled,2025-03-13 18:00:00
998,999,917,90,Completed,2025-03-01 15:31:00


# 2. Database Creation

In [732]:
import sqlite3

conn = sqlite3.connect("food_wastage.db")
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS providers")
cursor.execute("DROP TABLE IF EXISTS receivers")
cursor.execute("DROP TABLE IF EXISTS food_listings")
cursor.execute("DROP TABLE IF EXISTS claims")

# Recreate tables
cursor.execute('''
CREATE TABLE providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
)
''')

cursor.execute('''
CREATE TABLE receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    City TEXT,
    Contact TEXT
)
''')

cursor.execute('''
CREATE TABLE food_listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER,
    Expiry_Date DATE,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES providers (Provider_ID)
)
''')

cursor.execute('''
CREATE TABLE claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT,
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES food_listings (Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES receivers (Receiver_ID)
)
''')

conn.commit()


In [733]:
providers.to_sql("providers", conn, if_exists="append", index=False)
receivers.to_sql("receivers", conn, if_exists="append", index=False)
food_listings.to_sql("food_listings", conn, if_exists="append", index=False)
claims.to_sql("claims", conn, if_exists="append", index=False)


1000

In [734]:
import sqlite3

# Connect to local database file
conn = sqlite3.connect("food_wastage.db")
cursor = conn.cursor()

In [735]:
# Show all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:")
for table in tables:
    print(table[0])

Tables in database:
providers
receivers
food_listings
claims


In [736]:
# Display all data from a table
cursor.execute("SELECT * FROM providers;")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'gonzales-cochran', 'Supermarket', '74347 Christopher Extensions\nAndreamouth, OK 91839', 'new jessica', '+1-600-220-0480')
(2, 'nielsen johnson and fuller', 'Grocery Store', '91228 Hanson Stream\nWelchtown, OR 27136', 'east sheena', '+1-838-901-6297')
(3, 'miller-black', 'Supermarket', '561 Martinez Point Suite 507\nGuzmanchester, WA 94320', 'lake jesusview', '+1-517-295-2206')
(4, 'clark prince and williams', 'Grocery Store', '467 Bell Trail Suite 409\nPort Jesus, IA 61188', 'mendezmouth', '+1-944-893-5401')
(5, 'coleman-farley', 'Grocery Store', '078 Matthew Creek Apt. 319\nSaraborough, MA 53978', 'valentineside', '+1-193-714-6577')
(6, 'lawson-walters', 'Grocery Store', '1889 Barnes Gateway\nAdamview, ID 87971', 'shannonside', '+1-060-746-0544')
(7, 'ortiz-lee', 'Restaurant', '1842 Villarreal Shores\nWilliamfort, CT 44529', 'lake christopherburgh', '+1-563-889-7190')
(8, 'curtis-lewis', 'Supermarket', '4770 Miller Light Suite 260\nNew Charlesville, AR 97075', 'washingtonville',

In [737]:
# Loop through tables and get structure
for table in tables:
    table_name = table[0]
    print(f"\n=== Structure of '{table_name}' ===")
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    for col in columns:
        # col[1] = column name, col[2] = column type
        print(f"{col[1]} ({col[2]})")

conn.close()


=== Structure of 'providers' ===
Provider_ID (INTEGER)
Name (TEXT)
Type (TEXT)
Address (TEXT)
City (TEXT)
Contact (TEXT)

=== Structure of 'receivers' ===
Receiver_ID (INTEGER)
Name (TEXT)
Type (TEXT)
City (TEXT)
Contact (TEXT)

=== Structure of 'food_listings' ===
Food_ID (INTEGER)
Food_Name (TEXT)
Quantity (INTEGER)
Expiry_Date (DATE)
Provider_ID (INTEGER)
Provider_Type (TEXT)
Location (TEXT)
Food_Type (TEXT)
Meal_Type (TEXT)

=== Structure of 'claims' ===
Claim_ID (INTEGER)
Food_ID (INTEGER)
Receiver_ID (INTEGER)
Status (TEXT)
Timestamp (DATETIME)


# 3. Data Analysis

## SQL Queries

In [738]:
import sqlite3

# Open database connection
conn = sqlite3.connect("food_wastage.db")
cursor = conn.cursor()

### Food Providers & Receivers

#### Query 1: How many food providers and receivers are there in each city?

In [739]:
cursor = conn.cursor()

cursor.execute("""
SELECT p.City,
       COUNT(DISTINCT p.Provider_ID) AS Providers,
       COUNT(DISTINCT r.Receiver_ID) AS Receivers
FROM providers p
LEFT JOIN receivers r ON p.City = r.City
GROUP BY p.City;
""")

print("1. Providers & Receivers per city:", cursor.fetchall())

1. Providers & Receivers per city: [('adambury', 1, 0), ('adamsview', 1, 0), ('adamsville', 1, 0), ('aguirreville', 1, 0), ('alexanderchester', 1, 0), ('alexanderstad', 1, 0), ('allenborough', 1, 1), ('allenton', 1, 0), ('amandaborough', 1, 0), ('amandashire', 1, 0), ('amberton', 1, 0), ('ambertown', 1, 0), ('amyport', 1, 0), ('andersonmouth', 1, 0), ('andersonville', 1, 0), ('andreaborough', 1, 0), ('andrewsmouth', 1, 0), ('andrewstad', 1, 0), ('anitashire', 1, 0), ('annahaven', 1, 0), ('annetteburgh', 1, 0), ('anneville', 1, 0), ('anthonyborough', 1, 0), ('anthonychester', 1, 0), ('anthonyfort', 1, 0), ('anthonyhaven', 1, 0), ('anthonyshire', 1, 0), ('anthonyton', 1, 0), ('aprilberg', 1, 0), ('arnoldmouth', 1, 0), ('ashleyhaven', 1, 0), ('ashleyton', 1, 0), ('bairdfort', 1, 0), ('baldwinshire', 1, 0), ('barkerborough', 1, 0), ('barryside', 1, 0), ('bartonborough', 1, 0), ('basstown', 1, 0), ('batesstad', 1, 0), ('beasleyhaven', 1, 0), ('belindaville', 1, 0), ('bellport', 1, 0), ('ben

#### Query 2:  Which type of food provider contributes the most food?

In [740]:
cursor.execute("""
SELECT p.Type, SUM(f.Quantity) AS Total_Quantity
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Type
ORDER BY Total_Quantity DESC
LIMIT 1;
""")
print("2. Top provider type by contribution:", cursor.fetchall())

2. Top provider type by contribution: [('Restaurant', 6923)]


#### Query 3: Contact info of food providers in a specific city

In [741]:
cursor.execute("PRAGMA table_info(providers);")
columns = cursor.fetchall()
for col in columns:
    print(col)


(0, 'Provider_ID', 'INTEGER', 0, None, 1)
(1, 'Name', 'TEXT', 0, None, 0)
(2, 'Type', 'TEXT', 0, None, 0)
(3, 'Address', 'TEXT', 0, None, 0)
(4, 'City', 'TEXT', 0, None, 0)
(5, 'Contact', 'TEXT', 0, None, 0)


In [742]:
city_name = "adambury"
cursor.execute("""
SELECT Name, Contact
FROM providers
WHERE City = ?;
""", (city_name,))
print(f"3. Provider contacts in {city_name}:", cursor.fetchall())

3. Provider contacts in adambury: [('ibarra llc', '+1-670-338-0260')]


#### Query 4: Which receivers have claimed the most food?

In [743]:
cursor.execute("""
WITH claim_counts AS (
    SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims
    FROM receivers r
    JOIN claims c ON r.Receiver_ID = c.Receiver_ID
    GROUP BY r.Name
)
SELECT Name, Total_Claims
FROM claim_counts
WHERE Total_Claims = (SELECT MAX(Total_Claims) FROM claim_counts);
""")

print("Receivers with the highest number of claims:", cursor.fetchall())


Receivers with the highest number of claims: [('anthony garcia', 5), ('matthew webb', 5), ('scott hunter', 5), ('william frederick', 5)]


### Food Listings & Availability

#### Query 5: What is the total quantity of food available from all providers?



In [744]:
cursor.execute("""
SELECT SUM(Quantity) FROM food_listings;
""")
print("5. Total available food quantity:", cursor.fetchall())



5. Total available food quantity: [(25794,)]


#### Query 6: Which city has the highest number of food listings?


In [745]:
cursor.execute("""
SELECT p.City, COUNT(f.Food_ID) AS Listings
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.City
ORDER BY Listings DESC
LIMIT 1;
""")
print("6. City with most food listings:", cursor.fetchall())


6. City with most food listings: [('south kathryn', 6)]


#### Query 7: What are the most commonly available food types?

In [746]:
cursor.execute("""
SELECT Food_Type, COUNT(*) AS Count
FROM food_listings
GROUP BY Food_Type
ORDER BY Count DESC;
""")
print("7. Common food types:", cursor.fetchall())

7. Common food types: [('Vegetarian', 336), ('Vegan', 334), ('Non-Vegetarian', 330)]


### Claims & Distribution

#### Query 8: How many food claims for each food item?


In [747]:
cursor.execute("""
SELECT f.Food_Name, COUNT(c.Claim_ID) AS Claim_Count
FROM food_listings f
JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY f.Food_Name;
""")
print("8. Claims per food item:", cursor.fetchall())

8. Claims per food item: [('Bread', 94), ('Chicken', 102), ('Dairy', 110), ('Fish', 108), ('Fruits', 71), ('Pasta', 87), ('Rice', 122), ('Salad', 106), ('Soup', 114), ('Vegetables', 86)]


#### Query 9: Provider with the highest number of successful food claims


In [748]:
cursor.execute("""
SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
JOIN claims c ON f.Food_ID = c.Food_ID
WHERE c.Status = 'Completed'
GROUP BY p.Name
ORDER BY Successful_Claims DESC
LIMIT 1;
""")
print("9. Top provider by successful claims:", cursor.fetchall())

9. Top provider by successful claims: [('barry group', 5)]


#### Query 10: Percentage of claims by status


In [749]:
cursor.execute("""
SELECT Status,
       ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims)), 2) AS Percentage
FROM claims
GROUP BY Status;
""")
print("10. Claims percentage by status:", cursor.fetchall())

10. Claims percentage by status: [('Cancelled', 33.6), ('Completed', 33.9), ('Pending', 32.5)]


### Analysis & Insights

#### Query 11: Average quantity of food claimed per receiver


In [750]:
cursor.execute("""
SELECT r.Name, AVG(f.Quantity) AS Avg_Claimed
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY r.Name;
""")

print("11. Average quantity claimed per receiver:", cursor.fetchall())


11. Average quantity claimed per receiver: [('aaron keller', 39.0), ('aaron rios', 21.0), ('aaron scott', 45.0), ('abigail crawford', 25.666666666666668), ('adam browning', 5.0), ('adam moore', 27.333333333333332), ('aimee stone', 27.5), ('alan campbell', 6.0), ('alex leon', 14.0), ('alex rogers', 23.0), ('alex white', 18.0), ('alexander dean', 30.0), ('alexander mullins', 49.0), ('alexander walsh', 27.666666666666668), ('alexandra owens', 20.25), ('alexandra rogers', 30.0), ('alexis hayes', 12.0), ('alexis rodriguez', 8.0), ('alicia porter', 18.666666666666668), ('alicia thomas', 26.666666666666668), ('allison bray', 22.666666666666668), ('allison martinez', 10.0), ('allison pennington', 31.0), ('alvin day jr', 21.0), ('alvin west', 24.75), ('amanda brown', 8.0), ('amanda cain', 25.5), ('amanda kline', 1.0), ('amber arellano', 31.0), ('amber cruz', 18.0), ('amber hobbs', 26.0), ('amy barker', 41.0), ('amy brown', 34.0), ('amy perez', 32.5), ('amy thomas', 40.0), ('andrea hansen', 25.0

#### Query 12: Which meal type is claimed the most?


In [751]:
cursor.execute("""
SELECT f.Meal_Type, COUNT(c.Claim_ID) AS Claim_Count
FROM food_listings f
JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY f.Meal_Type
ORDER BY Claim_Count DESC;
""")
print("12. Most claimed meal type:", cursor.fetchall())


12. Most claimed meal type: [('Breakfast', 278), ('Lunch', 250), ('Snacks', 240), ('Dinner', 232)]


#### Query 13: Total quantity of food donated by each provider


In [752]:
cursor.execute("""
SELECT p.Name, SUM(f.Quantity) AS Total_Donated
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Name
ORDER BY Total_Donated DESC;
""")
print("13. Total food donated per provider:", cursor.fetchall())

13. Total food donated per provider: [('miller inc', 217), ('barry group', 179), ('evans wright and mitchell', 158), ('smith group', 150), ('campbell llc', 145), ('nelson llc', 142), ('ruiz-oneal', 140), ('blankenship-lewis', 124), ('kelly-ware', 123), ('bradford-martinez', 121), ('shepherd and sons', 116), ('hampton-lee', 116), ('jones ortega and rubio', 115), ('ortiz-lee', 114), ('johnson-ray', 113), ('barker llc', 110), ('moore group', 106), ('hill davis and stewart', 106), ('steele ltd', 104), ('butler-richardson', 104), ('lopez roach and roach', 102), ('hunter ballard and caldwell', 101), ('carter-jones', 101), ('hogan-johnston', 99), ('cox llc', 99), ('baker-mcdonald', 99), ('wong-reese', 98), ('jackson ltd', 98), ('aguilar-frederick', 98), ('schmidt-alexander', 97), ('phillips wolfe and martin', 97), ('garcia-hunter', 97), ('rogers harmon and gordon', 96), ('jones rojas and brown', 96), ('flores-wade', 96), ('clark prince and williams', 96), ('brown-stephens', 96), ('carey plc',

## Business Insights Using SQL Queries

#### Query 14: Which providers have donated the highest total quantity of food?

In [753]:
cursor.execute("""
SELECT p.Name, SUM(f.Quantity) AS Total_Donated
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Name
ORDER BY Total_Donated DESC;
""")
print("14. Providers with highest total quantity donated:\n", cursor.fetchall())


14. Providers with highest total quantity donated:
 [('miller inc', 217), ('barry group', 179), ('evans wright and mitchell', 158), ('smith group', 150), ('campbell llc', 145), ('nelson llc', 142), ('ruiz-oneal', 140), ('blankenship-lewis', 124), ('kelly-ware', 123), ('bradford-martinez', 121), ('shepherd and sons', 116), ('hampton-lee', 116), ('jones ortega and rubio', 115), ('ortiz-lee', 114), ('johnson-ray', 113), ('barker llc', 110), ('moore group', 106), ('hill davis and stewart', 106), ('steele ltd', 104), ('butler-richardson', 104), ('lopez roach and roach', 102), ('hunter ballard and caldwell', 101), ('carter-jones', 101), ('hogan-johnston', 99), ('cox llc', 99), ('baker-mcdonald', 99), ('wong-reese', 98), ('jackson ltd', 98), ('aguilar-frederick', 98), ('schmidt-alexander', 97), ('phillips wolfe and martin', 97), ('garcia-hunter', 97), ('rogers harmon and gordon', 96), ('jones rojas and brown', 96), ('flores-wade', 96), ('clark prince and williams', 96), ('brown-stephens', 96)

#### Query 15: Which cities have the highest number of providers contributing food?

In [754]:
# Get the maximum number of providers first
cursor.execute("""
SELECT MAX(Num_Providers)
FROM (
    SELECT COUNT(*) AS Num_Providers
    FROM providers
    GROUP BY City
);
""")
max_count = cursor.fetchone()[0]

# Then get all cities with that maximum count
cursor.execute("""
SELECT City, COUNT(*) AS Num_Providers
FROM providers
GROUP BY City
HAVING Num_Providers = ?
""", (max_count,))
print("\n15. Cities with the highest number of providers:\n", cursor.fetchall())



15. Cities with the highest number of providers:
 [('new carol', 3), ('south christopherborough', 3)]


#### Query 16: How many providers have never made a claim?



In [755]:
cursor.execute("""
SELECT p.Name
FROM providers p
LEFT JOIN food_listings f ON p.Provider_ID = f.Provider_ID
LEFT JOIN claims c ON f.Food_ID = c.Food_ID
WHERE c.Claim_ID IS NULL;
""")
print("\n16. Providers who never had any claims:\n", cursor.fetchall())


16. Providers who never had any claims:
 [('nielsen johnson and fuller',), ('miller-black',), ('coleman-farley',), ('coleman-farley',), ('lawson-walters',), ('ortiz-lee',), ('ortiz-lee',), ('curtis-lewis',), ('hall nguyen and martinez',), ('flores jackson and ford',), ('miller ltd',), ('parker flores and jennings',), ('nguyen-tran',), ('valdez-diaz',), ('miller ltd',), ('edwards-turner',), ('davis ltd',), ('cross group',), ('miller-carter',), ('johnson ltd',), ('gutierrez-palmer',), ('lester inc',), ('wright ltd',), ('rodriguez plc',), ('lawrence plc',), ('miranda wilcox and short',), ('cooper smith and king',), ('powell inc',), ('phelps-schmidt',), ('lawrence llc',), ('robinson plc',), ('kelly-ware',), ('martinez johnson and torres',), ('torres-perez',), ('fisher hernandez and webster',), ('wong-reese',), ('shaw inc',), ('wolfe and sons',), ('cook group',), ('arnold russo and foster',), ('mitchell campbell and jones',), ('mullen llc',), ('fletcher inc',), ('shannon ltd',), ('sherman-

#### Query 17: Which receivers have not claimed any food yet?

In [756]:
cursor.execute("""
SELECT r.Name
FROM receivers r
LEFT JOIN claims c ON r.Receiver_ID = c.Receiver_ID
WHERE c.Claim_ID IS NULL;
""")
print("\n17. Receivers who haven’t claimed any food:\n", cursor.fetchall())



17. Receivers who haven’t claimed any food:
 [('john romero',), ('james perez',), ('amber pacheco',), ('jamie sutton',), ('melissa castillo',), ('lisa williams',), ('courtney sanders',), ('george palmer',), ('emily davidson',), ('kristina meza',), ('kurt ward',), ('tammy moore',), ('lisa mcclain',), ('nicholas robinson',), ('bruce sanchez',), ('teresa davis',), ('john ramirez',), ('abigail davis',), ('julia elliott',), ('kevin lee',), ('gabriel nicholson',), ('timothy clark',), ('veronica howard',), ('erika rodriguez',), ('jessica rivas',), ('john vazquez',), ('debra sheppard',), ('rodney walker',), ('carolyn whitaker',), ('timothy daniel',), ('kimberly mills',), ('anna lopez',), ('aaron davis',), ('meagan ramirez',), ('eric jackson',), ('theresa ramirez',), ('laura bennett',), ('julie hines',), ('deanna lewis',), ('jeffery mueller',), ('mr david lambert',), ('ashley james',), ('bridget peterson',), ('anthony green',), ('david johnson',), ('suzanne dawson',), ('karla castillo',), ('de

#### Query 18: How many providers have not claimed any food yet?


In [757]:
cursor.execute("""
SELECT p.Name
FROM providers p
LEFT JOIN food_listings f ON p.Provider_ID = f.Provider_ID
LEFT JOIN claims c ON f.Food_ID = c.Food_ID
WHERE c.Claim_ID IS NULL;
""")
print("\n1. Providers who never made a claim:\n", cursor.fetchall())


1. Providers who never made a claim:
 [('nielsen johnson and fuller',), ('miller-black',), ('coleman-farley',), ('coleman-farley',), ('lawson-walters',), ('ortiz-lee',), ('ortiz-lee',), ('curtis-lewis',), ('hall nguyen and martinez',), ('flores jackson and ford',), ('miller ltd',), ('parker flores and jennings',), ('nguyen-tran',), ('valdez-diaz',), ('miller ltd',), ('edwards-turner',), ('davis ltd',), ('cross group',), ('miller-carter',), ('johnson ltd',), ('gutierrez-palmer',), ('lester inc',), ('wright ltd',), ('rodriguez plc',), ('lawrence plc',), ('miranda wilcox and short',), ('cooper smith and king',), ('powell inc',), ('phelps-schmidt',), ('lawrence llc',), ('robinson plc',), ('kelly-ware',), ('martinez johnson and torres',), ('torres-perez',), ('fisher hernandez and webster',), ('wong-reese',), ('shaw inc',), ('wolfe and sons',), ('cook group',), ('arnold russo and foster',), ('mitchell campbell and jones',), ('mullen llc',), ('fletcher inc',), ('shannon ltd',), ('sherman-wol

#### Query 19: Which providers consistently list food items with near-expiry dates?


In [758]:
cursor.execute("""
SELECT p.Name, f.Food_Name, f.Expiry_Date
FROM food_listings f
JOIN providers p ON f.Provider_ID = p.Provider_ID
WHERE DATE(f.Expiry_Date) <= DATE('now', '+7 days')
ORDER BY f.Expiry_Date ASC;
""")
print("\n19. Providers listing near-expiry items (within 7 days):\n", cursor.fetchall())



19. Providers listing near-expiry items (within 7 days):
 [('cannon-garcia', 'Fruits', '2025-03-16'), ('moore-flowers', 'Soup', '2025-03-16'), ('lucas bush and miller', 'Rice', '2025-03-16'), ('powers clark and snyder', 'Rice', '2025-03-16'), ('gillespie foster and boyd', 'Fruits', '2025-03-16'), ('small ltd', 'Bread', '2025-03-16'), ('maldonado-davis', 'Rice', '2025-03-16'), ('ayers llc', 'Pasta', '2025-03-16'), ('jackson ltd', 'Pasta', '2025-03-16'), ('young-luna', 'Rice', '2025-03-16'), ('williams inc', 'Soup', '2025-03-16'), ('fernandez-bridges', 'Rice', '2025-03-16'), ('miller inc', 'Chicken', '2025-03-16'), ('carter-jones', 'Bread', '2025-03-16'), ('wong-reese', 'Rice', '2025-03-16'), ('yoder-murray', 'Vegetables', '2025-03-16'), ('barnes castro and curtis', 'Fruits', '2025-03-16'), ('schmidt-alexander', 'Rice', '2025-03-16'), ('conway-barr', 'Chicken', '2025-03-16'), ('johnson group', 'Bread', '2025-03-16'), ('nichols-barnes', 'Salad', '2025-03-16'), ('lambert ltd', 'Dairy', '2

#### Query 20: How many food items are expiring within the next 7 days?


In [759]:
cursor.execute("""
SELECT Food_Name, Expiry_Date
FROM food_listings
WHERE DATE(Expiry_Date) <= DATE('now', '+7 days')
ORDER BY Expiry_Date ASC;
""")
print("\n20. Food items expiring within next 7 days:\n", cursor.fetchall())


20. Food items expiring within next 7 days:
 [('Fruits', '2025-03-16'), ('Soup', '2025-03-16'), ('Rice', '2025-03-16'), ('Rice', '2025-03-16'), ('Fruits', '2025-03-16'), ('Bread', '2025-03-16'), ('Rice', '2025-03-16'), ('Pasta', '2025-03-16'), ('Pasta', '2025-03-16'), ('Rice', '2025-03-16'), ('Soup', '2025-03-16'), ('Rice', '2025-03-16'), ('Chicken', '2025-03-16'), ('Bread', '2025-03-16'), ('Rice', '2025-03-16'), ('Vegetables', '2025-03-16'), ('Fruits', '2025-03-16'), ('Rice', '2025-03-16'), ('Chicken', '2025-03-16'), ('Bread', '2025-03-16'), ('Salad', '2025-03-16'), ('Dairy', '2025-03-16'), ('Salad', '2025-03-16'), ('Fruits', '2025-03-16'), ('Fruits', '2025-03-16'), ('Dairy', '2025-03-16'), ('Chicken', '2025-03-16'), ('Fruits', '2025-03-16'), ('Rice', '2025-03-16'), ('Soup', '2025-03-16'), ('Vegetables', '2025-03-16'), ('Salad', '2025-03-16'), ('Fish', '2025-03-16'), ('Fish', '2025-03-16'), ('Bread', '2025-03-16'), ('Soup', '2025-03-16'), ('Pasta', '2025-03-16'), ('Dairy', '2025-03-1

#### Query 21: Which meal types are most frequently listed in each city?

In [760]:
cursor.execute("""
SELECT Location, Meal_Type, COUNT(*) AS Count
FROM food_listings
GROUP BY Location, Meal_Type
ORDER BY Location, Count DESC;
""")
print("\n21. Most frequently listed meal types per city:\n", cursor.fetchall())


21. Most frequently listed meal types per city:
 [('Adambury', 'Lunch', 1), ('Alexanderchester', 'Dinner', 1), ('Allenborough', 'Dinner', 1), ('Allenton', 'Snacks', 1), ('Allenton', 'Lunch', 1), ('Allenton', 'Breakfast', 1), ('Amandashire', 'Breakfast', 1), ('Amberton', 'Lunch', 1), ('Amberton', 'Dinner', 1), ('Andersonmouth', 'Breakfast', 1), ('Andersonville', 'Snacks', 1), ('Andreaborough', 'Snacks', 1), ('Andreaborough', 'Dinner', 1), ('Andrewsmouth', 'Dinner', 1), ('Annahaven', 'Lunch', 1), ('Anneville', 'Lunch', 3), ('Anthonyborough', 'Snacks', 1), ('Anthonyhaven', 'Snacks', 1), ('Anthonyshire', 'Dinner', 1), ('Anthonyton', 'Snacks', 1), ('Aprilberg', 'Lunch', 1), ('Ashleyhaven', 'Snacks', 1), ('Bairdfort', 'Snacks', 1), ('Bartonborough', 'Snacks', 1), ('Bartonborough', 'Lunch', 1), ('Basstown', 'Lunch', 1), ('Basstown', 'Breakfast', 1), ('Batesstad', 'Snacks', 1), ('Belindaville', 'Lunch', 1), ('Belindaville', 'Breakfast', 1), ('Bellport', 'Snacks', 1), ('Bellport', 'Lunch', 1),

#### Query 22: Which receivers claim the most diverse types of food (variety of Food_Type)?


In [761]:
cursor.execute("""
SELECT MAX(Food_Variety)
FROM (
    SELECT r.Name, COUNT(DISTINCT f.Food_Type) AS Food_Variety
    FROM receivers r
    JOIN claims c ON r.Receiver_ID = c.Receiver_ID
    JOIN food_listings f ON c.Food_ID = f.Food_ID
    GROUP BY r.Name
);
""")
max_variety = cursor.fetchone()[0]

# Then get all receivers with that maximum variety
cursor.execute("""
SELECT r.Name, COUNT(DISTINCT f.Food_Type) AS Food_Variety
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY r.Name
HAVING Food_Variety = ?
""", (max_variety,))
print("\n22. Receivers claiming most diverse food types:\n", cursor.fetchall())



22. Receivers claiming most diverse food types:
 [('abigail crawford', 3), ('alexander walsh', 3), ('alexandra rogers', 3), ('alicia porter', 3), ('allison pennington', 3), ('anna buckley', 3), ('arthur thomas', 3), ('bruce alexander', 3), ('christine davis', 3), ('donald caldwell', 3), ('donna hughes', 3), ('james brown', 3), ('james miller', 3), ('jennifer nelson', 3), ('justin powell', 3), ('kimberly nelson', 3), ('matthew tran', 3), ('michael walls', 3), ('oscar bauer', 3), ('samuel lewis', 3), ('sarah carter', 3), ('scott hunter', 3), ('scott schroeder', 3), ('vanessa johnson', 3), ('victor crawford', 3), ('victor diaz', 3), ('william frederick', 3)]


#### Query 23: What is the ratio of claimed vs. unclaimed food per provider?


In [762]:
cursor.execute("""
SELECT p.Name,
       SUM(CASE WHEN c.Claim_ID IS NOT NULL THEN 1 ELSE 0 END) AS Claimed,
       SUM(CASE WHEN c.Claim_ID IS NULL THEN 1 ELSE 0 END) AS Unclaimed
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
LEFT JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY p.Name;
""")
print("\n23. Claimed vs unclaimed food per provider:\n", cursor.fetchall())


23. Claimed vs unclaimed food per provider:
 [('abbott brooks and moreno', 1, 0), ('abbott-robinson', 0, 1), ('adams-meyer', 0, 1), ('adams-young', 0, 1), ('aguilar group', 0, 2), ('aguilar-frederick', 3, 1), ('alexander plc', 2, 0), ('alexander-mendoza', 1, 0), ('allen llc', 0, 1), ('allen ramsey and cortez', 2, 0), ('allen zimmerman and harrington', 2, 0), ('allen-mccullough', 7, 0), ('alvarado llc', 2, 0), ('anderson adams and roberts', 4, 1), ('anderson and sons', 0, 1), ('anderson nicholson and bruce', 3, 1), ('anderson patrick and scott', 1, 0), ('anderson robinson and phillips', 1, 0), ('andrews inc', 0, 1), ('andrews-wiley', 1, 0), ('arroyo-henderson', 1, 0), ('atkins group', 2, 0), ('austin griffin and ward', 2, 0), ('austin-yu', 1, 0), ('avery plc', 1, 0), ('avila group', 2, 0), ('ayala llc', 3, 0), ('ayers llc', 0, 2), ('ayers-lewis', 0, 1), ('baird plc', 1, 2), ('baker gardner and callahan', 1, 0), ('baker group', 2, 0), ('baker llc', 2, 1), ('baker-mcdonald', 2, 2), ('bal

#### Query 24: Which providers have the highest percentage of canceled claims?


In [763]:
cursor.execute("""
SELECT MAX(Cancelled_Percentage)
FROM (
    SELECT p.Name,
           ROUND(100.0 * SUM(CASE WHEN c.Status = 'Cancelled' THEN 1 ELSE 0 END)/COUNT(c.Claim_ID),2) AS Cancelled_Percentage
    FROM providers p
    JOIN food_listings f ON p.Provider_ID = f.Provider_ID
    JOIN claims c ON f.Food_ID = c.Food_ID
    GROUP BY p.Name
);
""")
max_percentage = cursor.fetchone()[0]

cursor.execute("""
SELECT p.Name,
       COUNT(c.Claim_ID) AS Total_Claims,
       SUM(CASE WHEN c.Status = 'Cancelled' THEN 1 ELSE 0 END) AS Cancelled_Claims,
       ROUND(100.0 * SUM(CASE WHEN c.Status = 'Cancelled' THEN 1 ELSE 0 END)/COUNT(c.Claim_ID),2) AS Cancelled_Percentage
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY p.Name
HAVING Cancelled_Percentage = ?
""", (max_percentage,))
print("\n24. Providers with highest percentage of canceled claims:\n", cursor.fetchall())



24. Providers with highest percentage of canceled claims:
 [('andrews-wiley', 1, 1, 100.0), ('baird plc', 1, 1, 100.0), ('baker gardner and callahan', 1, 1, 100.0), ('baker group', 2, 2, 100.0), ('barber ltd', 3, 3, 100.0), ('barker gibbs and lopez', 2, 2, 100.0), ('barker llc', 2, 2, 100.0), ('barrera-ward', 2, 2, 100.0), ('beck group', 1, 1, 100.0), ('blair group', 1, 1, 100.0), ('brooks-mccarty', 1, 1, 100.0), ('bryant jacobson and west', 1, 1, 100.0), ('buck-carey', 1, 1, 100.0), ('callahan-olsen', 1, 1, 100.0), ('cole-stein', 2, 2, 100.0), ('cox-mcintyre', 3, 3, 100.0), ('daniel schroeder and morse', 1, 1, 100.0), ('davis hughes and leblanc', 1, 1, 100.0), ('davis-hurley', 1, 1, 100.0), ('davis-kennedy', 1, 1, 100.0), ('foster ltd', 1, 1, 100.0), ('gonzales group', 1, 1, 100.0), ('gonzales-keith', 1, 1, 100.0), ('harris santos and roberts', 1, 1, 100.0), ('henry plc', 3, 3, 100.0), ('house james and miller', 1, 1, 100.0), ('jackson gonzalez and smith', 1, 1, 100.0), ('james group

#### Query 25: How many food items are claimed multiple times by different receivers?

In [764]:
cursor.execute("""
SELECT f.Food_Name, COUNT(DISTINCT c.Receiver_ID) AS Num_Receivers
FROM food_listings f
JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY f.Food_Name
HAVING Num_Receivers > 1
ORDER BY Num_Receivers DESC;
""")
print("\n25. Food items claimed multiple times by different receivers:\n", cursor.fetchall())



25. Food items claimed multiple times by different receivers:
 [('Rice', 112), ('Dairy', 107), ('Soup', 105), ('Fish', 102), ('Salad', 99), ('Chicken', 97), ('Bread', 88), ('Vegetables', 84), ('Pasta', 80), ('Fruits', 70)]


# 4.  Application Development

In [765]:
# Required packages
!pip install streamlit pandas sqlite3
!pip install streamlit


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [766]:
!pip install pyngrok
!pip install streamlit

In [767]:
!ngrok authtoken 31HmOZUzJ2dvSKPHfx9VB29Nhu3_5QQiseh7xAGXFWpYmVsxb

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [768]:
# Install required packages
!pip install streamlit pyngrok pandas


In [769]:
# --- CONNECT TO DATABASE ---
conn = sqlite3.connect("food_wastage.db")
c = conn.cursor()

In [770]:
%%writefile food_dashboard.py
import streamlit as st
import pandas as pd
import sqlite3
import datetime

# --- Connect to database ---
conn = sqlite3.connect("food_wastage.db")
c = conn.cursor()

def show_dashboard():
    st.title("🍽️ Food Donation & Claims Dashboard")
    st.write("📊 Dashboard powered by Streamlit & SQLite")

    # --- 1. Providers & Receivers per City ---
    providers_receivers = pd.read_sql("""
    SELECT p.City,
           COUNT(DISTINCT p.Provider_ID) AS Providers,
           COUNT(DISTINCT r.Receiver_ID) AS Receivers
    FROM providers p
    LEFT JOIN receivers r ON p.City = r.City
    GROUP BY p.City;
    """, conn)
    st.subheader("1️⃣ Providers & Receivers per City")
    st.dataframe(providers_receivers)
    st.bar_chart(providers_receivers.set_index('City'))

    # --- 2. Top provider type by contribution ---
    top_provider_type = pd.read_sql("""
    SELECT Provider_Type, SUM(Quantity) AS Total_Quantity
    FROM food_listings
    GROUP BY Provider_Type
    ORDER BY Total_Quantity DESC
    LIMIT 1;
    """, conn)
    st.subheader("2️⃣ Top Provider Type by Contribution")
    st.table(top_provider_type)

    # --- 3. Provider contacts in a city ---
    city_name = st.text_input("Enter city for provider contacts", "adambury")
    provider_contacts = pd.read_sql("""
    SELECT Name, Contact
    FROM providers
    WHERE City = ?;
    """, conn, params=(city_name,))
    st.subheader(f"3️⃣ Provider Contacts in {city_name.title()}")
    st.dataframe(provider_contacts)

    # --- 4. Receivers with highest claims ---
    top_receivers = pd.read_sql("""
    WITH claim_counts AS (
        SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims
        FROM receivers r
        JOIN claims c ON r.Receiver_ID = c.Receiver_ID
        GROUP BY r.Name
    )
    SELECT Name, Total_Claims
    FROM claim_counts
    WHERE Total_Claims = (SELECT MAX(Total_Claims) FROM claim_counts);
    """, conn)
    st.subheader("4️⃣ Receivers with Highest Number of Claims")
    st.table(top_receivers)

    # --- 5. Total available food quantity ---
    total_food = pd.read_sql("SELECT SUM(Quantity) AS Total_Quantity FROM food_listings;", conn)
    st.subheader("5️⃣ Total Food Available")
    st.metric("Total Quantity", total_food['Total_Quantity'].iloc[0])

    # --- 6. City with most food listings ---
    top_city_listings = pd.read_sql("""
    SELECT Location, COUNT(Food_ID) AS Listings
    FROM food_listings
    GROUP BY Location
    ORDER BY Listings DESC
    LIMIT 1;
    """, conn)
    st.subheader("6️⃣ City with Most Food Listings")
    st.table(top_city_listings)

    # --- 7. Most common food types ---
    food_types = pd.read_sql("""
    SELECT Food_Type, COUNT(*) AS Count
    FROM food_listings
    GROUP BY Food_Type
    ORDER BY Count DESC;
    """, conn)
    st.subheader("7️⃣ Most Common Food Types")
    st.bar_chart(food_types.set_index('Food_Type'))

    # --- 8. Claims per food item ---
    claims_per_item = pd.read_sql("""
    SELECT f.Food_Name, COUNT(c.Claim_ID) AS Claim_Count
    FROM food_listings f
    JOIN claims c ON f.Food_ID = c.Food_ID
    GROUP BY f.Food_Name;
    """, conn)
    st.subheader("8️⃣ Claims Per Food Item")
    st.dataframe(claims_per_item)

    # --- 9. Provider with highest successful claims ---
    top_provider_claims = pd.read_sql("""
    SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
    FROM providers p
    JOIN food_listings f ON p.Provider_ID = f.Provider_ID
    JOIN claims c ON f.Food_ID = c.Food_ID
    WHERE c.Status = 'Completed'
    GROUP BY p.Name
    ORDER BY Successful_Claims DESC
    LIMIT 1;
    """, conn)
    st.subheader("9️⃣ Top Provider by Successful Claims")
    st.table(top_provider_claims)

    # --- 10. Claims percentage by status ---
    claims_status = pd.read_sql("""
    SELECT Status,
           ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims)), 2) AS Percentage
    FROM claims
    GROUP BY Status;
    """, conn)
    st.subheader("🔟 Claims Percentage by Status")
    st.dataframe(claims_status)
    st.bar_chart(claims_status.set_index('Status'))

    # --- 11. Average quantity claimed per receiver ---
    avg_claim_per_receiver = pd.read_sql("""
    SELECT r.Name, AVG(f.Quantity) AS Avg_Claimed
    FROM receivers r
    JOIN claims c ON r.Receiver_ID = c.Receiver_ID
    JOIN food_listings f ON c.Food_ID = f.Food_ID
    GROUP BY r.Name;
    """, conn)
    st.subheader("1️⃣1️⃣ Average Quantity Claimed Per Receiver")
    st.dataframe(avg_claim_per_receiver)

    # --- 12. Most claimed meal type ---
    most_claimed_meal = pd.read_sql("""
    SELECT Meal_Type, COUNT(c.Claim_ID) AS Claim_Count
    FROM food_listings f
    JOIN claims c ON f.Food_ID = c.Food_ID
    GROUP BY Meal_Type
    ORDER BY Claim_Count DESC;
    """, conn)
    st.subheader("1️⃣2️⃣ Most Claimed Meal Type")
    st.bar_chart(most_claimed_meal.set_index('Meal_Type'))

    # --- 13. Total food donated per provider ---
    total_donated = pd.read_sql("""
    SELECT p.Name, SUM(f.Quantity) AS Total_Donated
    FROM providers p
    JOIN food_listings f ON p.Provider_ID = f.Provider_ID
    GROUP BY p.Name
    ORDER BY Total_Donated DESC;
    """, conn)
    st.subheader("1️⃣3️⃣ Total Food Donated Per Provider")
    st.dataframe(total_donated)

    # --- 14. Food Items Near Expiry (Next 3 Days) & 15: Optional additional analysis ---
    near_expiry_items = pd.read_sql("""
    SELECT Food_Name, Expiry_Date, Quantity
    FROM food_listings
    WHERE DATE(Expiry_Date) <= DATE('now', '+3 day');
    """, conn)
    st.subheader("1️⃣4️⃣ Food Items Near Expiry (Next 3 Days)")
    st.dataframe(near_expiry_items)

    # --- 15. Top 10 Cities by Food Claims ---
    top_claiming_cities = pd.read_sql("""
    SELECT Location, COUNT(c.Claim_ID) AS Total_Claims
    FROM food_listings f
    JOIN claims c ON f.Food_ID = c.Food_ID
    GROUP BY Location
    ORDER BY Total_Claims DESC
    LIMIT 10;
    """, conn)
    st.subheader("1️⃣5️⃣ Top 10 Cities by Food Claims")
    st.bar_chart(top_claiming_cities.set_index('Location'))
    st.write("📊 Dashboard powered by Streamlit & SQLite")

def show_list_food():
    st.title("➕ List New Food Surplus")
    st.markdown("---")
    providers = pd.read_sql("SELECT Provider_ID, Name, City, Type FROM providers;", conn)
    provider_names = providers['Name'].tolist()

    with st.form("list_food_form"):
        st.subheader("Food Item Details")
        provider_name = st.selectbox("Select Provider:", provider_names)
        provider_info = providers[providers['Name'] == provider_name]

        if not provider_info.empty:
            provider_id = int(provider_info['Provider_ID'].iloc[0])
            provider_type = provider_info['Type'].iloc[0]
            location = provider_info['City'].iloc[0]

            food_name = st.text_input("Food Name:")
            food_type = st.selectbox("Food Type:", ["Vegetables", "Fruits", "Prepared Meal", "Snacks", "Bakery", "Dairy"])
            quantity = st.number_input("Quantity:", min_value=1, step=1)
            expiry_date = st.date_input("Expiry Date:", min_value=datetime.date.today())
            meal_type = st.selectbox("Meal Type:", ["Breakfast", "Lunch", "Dinner", "Snack", "Any"])

            submitted = st.form_submit_button("List Food")

            if submitted:
                if food_name and quantity:
                    c.execute("""
                        INSERT INTO food_listings (Provider_ID, Provider_Type, Location, Food_Name, Food_Type, Quantity, Expiry_Date, Meal_Type)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    """, (provider_id, provider_type, location, food_name, food_type, quantity, str(expiry_date), meal_type))
                    conn.commit()
                    st.success("🎉 Food item listed successfully!")
                else:
                    st.error("Please fill in all the details.")
        else:
            st.info("Please add a provider first using the 'Add Provider' page.")

def show_claim_food():
    st.title("🛒 Claim Available Food")
    st.markdown("---")

    receivers = pd.read_sql("SELECT Receiver_ID, Name FROM receivers;", conn)
    receiver_names = receivers['Name'].tolist()

    available_food = pd.read_sql("""
        SELECT Food_ID, Food_Name, Quantity, Expiry_Date, Provider_Type, Location
        FROM food_listings
        WHERE Food_ID NOT IN (SELECT Food_ID FROM claims WHERE Status IN ('Pending', 'Completed'))
        AND DATE(Expiry_Date) > DATE('now')
    """, conn)

    if available_food.empty:
        st.info("No food items are currently available for claiming.")
        return

    with st.form("claim_food_form"):
        st.subheader("Claim Details")
        receiver_name = st.selectbox("Select Receiver:", receiver_names)
        receiver_id = receivers[receivers['Name'] == receiver_name]['Receiver_ID'].iloc[0]

        food_options = available_food.apply(
            lambda row: f"{row['Food_Name']} from {row['Provider_Type']} in {row['Location']} (Qty: {row['Quantity']})", axis=1
        ).tolist()

        selected_food_str = st.selectbox("Select Food to Claim:", food_options)

        selected_food_id = None
        if selected_food_str:
            selected_food_id = available_food[
                available_food.apply(
                    lambda row: f"{row['Food_Name']} from {row['Provider_Type']} in {row['Location']} (Qty: {row['Quantity']})", axis=1
                ) == selected_food_str
            ]['Food_ID'].iloc[0]

        submitted = st.form_submit_button("Claim Food")

        if submitted:
            if selected_food_id:
                c.execute("""
                    INSERT INTO claims (Food_ID, Receiver_ID, Status, Timestamp)
                    VALUES (?, ?, ?, ?)
                """, (selected_food_id, receiver_id, 'Pending', datetime.datetime.now()))
                conn.commit()
                st.success("✅ Food claimed successfully! Awaiting provider confirmation.")
            else:
                st.error("Please select a food item to claim.")

def show_add_provider():
    st.title("➕ Add New Provider")
    st.markdown("---")
    with st.form("add_provider_form"):
        st.subheader("Provider Details")
        name = st.text_input("Name:")
        contact = st.text_input("Contact:")
        address = st.text_input("Address:")
        city = st.text_input("City:")
        provider_type = st.selectbox("Type:", ["Restaurant", "Bakery", "Supermarket", "Grocery Store", "Catering Service"])

        submitted = st.form_submit_button("Add Provider")

        if submitted:
            if name and contact and city and provider_type:
                c.execute("""
                    INSERT INTO providers (Name, Contact, Address, City, Type)
                    VALUES (?, ?, ?, ?, ?)
                """, (name, contact, address, city, provider_type))
                conn.commit()
                st.success(f"✅ Provider '{name}' added successfully!")
            else:
                st.error("Please fill in all the details.")

def show_add_receiver():
    st.title("➕ Add New Receiver")
    st.markdown("---")
    with st.form("add_receiver_form"):
        st.subheader("Receiver Details")
        name = st.text_input("Name:")
        contact = st.text_input("Contact:")
        city = st.text_input("City:")
        receiver_type = st.selectbox("Type:", ["Non-profit", "Community Center", "Individual", "Other"])

        submitted = st.form_submit_button("Add Receiver")

        if submitted:
            if name and contact and city and receiver_type:
                c.execute("""
                    INSERT INTO receivers (Name, Contact, City, Type)
                    VALUES (?, ?, ?, ?)
                """, (name, contact, city, receiver_type))
                conn.commit()
                st.success(f"✅ Receiver '{name}' added successfully!")
            else:
                st.error("Please fill in all the details.")

# --- Main App Logic ---
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to:", ["Dashboard", "List Food", "Claim Food", "Add Provider", "Add Receiver"])

if page == "Dashboard":
    show_dashboard()
elif page == "List Food":
    show_list_food()
elif page == "Claim Food":
    show_claim_food()
elif page == "Add Provider":
    show_add_provider()
elif page == "Add Receiver":
    show_add_receiver()

conn.close()

Overwriting food_dashboard.py


In [771]:
from pyngrok import ngrok
import os
import time

# ngrok authentication token (replace with your actual token)
# ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")

# Kill any existing ngrok tunnels and processes to be sure
try:
    ngrok.kill()
    time.sleep(2) # Wait for a moment to ensure processes are terminated
except Exception as e:
    print(f"Ngrok kill failed: {e}")

# Start Streamlit in the background
os.system("streamlit run food_dashboard.py --server.port=8501 &")

# Create ngrok tunnel
try:
    public_url = ngrok.connect(8501)
    print("ngrok tunnel is live at:", public_url)
except Exception as e:
    print(f"An error occurred while creating the ngrok tunnel: {e}")

ngrok tunnel is live at: NgrokTunnel: "https://d14f7d203bf4.ngrok-free.app" -> "http://localhost:8501"


# 5. Deployment

Install Required Packages
Installed Streamlit, Pandas, and Pyngrok to build the dashboard and expose it publicly while running in Colab.

- Upload Database
Uploaded the SQLite database food_wastage.db to Colab so the app could access provider, receiver, and food listing data.

- Create Streamlit App
Developed food_dashboard.py with 15 SQL queries to analyze food donations, claims, and trends. Added tables, charts, and metrics for visualization.

- Run Streamlit in Colab
Started the Streamlit app in the background and used ngrok to generate a public URL for real-time access and interaction.

- Push to GitHub
Created a GitHub repository with the app file, database, and requirements.txt to list dependencies.

- Deploy on Streamlit Community Cloud
Connected Streamlit Cloud to the GitHub repo, selected the main file (food_dashboard.py), and clicked Deploy. Streamlit Cloud installed dependencies and launched the app automatically.

- Access Live Dashboard
The app became publicly accessible via a URL:
https://local-food-wastage-management-system-cvpx32dpmxdz4q5rmzzkhy.streamlit.app/
This allows anyone to view and interact with the dashboard in real-time.